# Weather maps

https://azure.microsoft.com/en-us/products/azure-maps/?msockid=2e39c66c693c66a5151fd200687567d0

https://learn.microsoft.com/en-us/azure/azure-maps/

In [1]:
import datetime
import json
import os
import requests
import sys

from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Dict, List, Optional, Tuple

In [2]:
load_dotenv("azure.env")

True

In [3]:
sys.version

'3.10.14 (main, May  6 2024, 19:42:50) [GCC 11.2.0]'

## Settings

In [4]:
class AzureMapsWeather:
    """
    Azure Maps Weather API client for weather forecasting along routes and locations.
    """
    
    def __init__(self, subscription_key: str):
        """
        Initialize the Azure Maps Weather client.
        
        Args:
            subscription_key: Your Azure Maps subscription key
        """
        self.subscription_key = subscription_key
        self.base_url = "https://atlas.microsoft.com/weather"
        
    def get_current_conditions(self, latitude: float, longitude: float) -> Dict:
        """
        Get current weather conditions for a specific location.
        
        Args:
            latitude: Latitude of the location
            longitude: Longitude of the location
            
        Returns:
            Dictionary containing current weather conditions
        """
        url = f"{self.base_url}/currentConditions/json"
        
        params = {
            "subscription-key": self.subscription_key,
            "api-version": "1.0",
            "query": f"{latitude},{longitude}"
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching current conditions: {e}")
            return {}
    
    def get_daily_forecast(
        self, 
        latitude: float, 
        longitude: float, 
        duration: int = 5
    ) -> Dict:
        """
        Get daily weather forecast for a location.
        
        Args:
            latitude: Latitude of the location
            longitude: Longitude of the location
            duration: Number of days to forecast (1-15)
            
        Returns:
            Dictionary containing daily forecast data
        """
        url = f"{self.base_url}/forecast/daily/json"
        
        params = {
            "subscription-key": self.subscription_key,
            "api-version": "1.0",
            "query": f"{latitude},{longitude}",
            "duration": duration
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching daily forecast: {e}")
            return {}
    
    def get_hourly_forecast(
        self, 
        latitude: float, 
        longitude: float, 
        duration: int = 12
    ) -> Dict:
        """
        Get hourly weather forecast for a location.
        
        Args:
            latitude: Latitude of the location
            longitude: Longitude of the location
            duration: Number of hours to forecast (1-240)
            
        Returns:
            Dictionary containing hourly forecast data
        """
        url = f"{self.base_url}/forecast/hourly/json"
        
        params = {
            "subscription-key": self.subscription_key,
            "api-version": "1.0",
            "query": f"{latitude},{longitude}",
            "duration": duration
        }
        
        try:
            response = requests.get(url, params=params)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"Error fetching hourly forecast: {e}")
            return {}

## Examples

In [5]:
weather_client = AzureMapsWeather(os.getenv('AZURE_MAPS_KEY'))

In [6]:
# Address to geocode
address = 'Paris, France'

url = 'https://atlas.microsoft.com/search/address/json'

params = {
    'api-version': '1.0',
    'subscription-key': os.getenv('AZURE_MAPS_KEY'),
    'query': address
}

response = requests.get(url, params=params)
# Parse the response
if response.status_code == 200:
    data = response.json()
    position = data['results'][0]['position']
    lat = position['lat']
    lon = position['lon']
    print(f"Latitude: {lat}, Longitude: {lon} for address: {address}")
else:
    print(f"Error: {response.status_code}, {response.text}")

Latitude: 48.856895, Longitude: 2.3508487 for address: Paris, France


### Current weather data

In [7]:
test_weather = weather_client.get_current_conditions(lat, lon)
print(f"Current weather data for {address}")
test_weather

Current weather data for Paris, France


{'results': [{'dateTime': '2025-09-01T09:56:00+02:00',
   'phrase': 'Cloudy',
   'iconCode': 7,
   'hasPrecipitation': False,
   'isDayTime': True,
   'temperature': {'value': 17.0, 'unit': 'C', 'unitType': 17},
   'realFeelTemperature': {'value': 16.3, 'unit': 'C', 'unitType': 17},
   'realFeelTemperatureShade': {'value': 15.2, 'unit': 'C', 'unitType': 17},
   'relativeHumidity': 82,
   'dewPoint': {'value': 13.9, 'unit': 'C', 'unitType': 17},
   'wind': {'direction': {'degrees': 203.0, 'localizedDescription': 'SSW'},
    'speed': {'value': 15.8, 'unit': 'km/h', 'unitType': 7}},
   'windGust': {'speed': {'value': 25.8, 'unit': 'km/h', 'unitType': 7}},
   'uvIndex': 1,
   'uvIndexPhrase': 'Low',
   'visibility': {'value': 24.1, 'unit': 'km', 'unitType': 6},
   'obstructionsToVisibility': '',
   'cloudCover': 91,
   'ceiling': {'value': 6096.0, 'unit': 'm', 'unitType': 5},
   'pressure': {'value': 1009.0, 'unit': 'mb', 'unitType': 14},
   'pressureTendency': {'localizedDescription': 'Ri

## Daily forecasts

In [8]:
print(f"Daily forecast data for {address}:\n")
daily_forecast = weather_client.get_daily_forecast(lat, lon, duration=5)

print(json.dumps(daily_forecast, indent=5))

Daily forecast data for Paris, France:

{
     "summary": {
          "startDate": "2025-09-01T08:00:00+02:00",
          "endDate": "2025-09-01T14:00:00+02:00",
          "severity": 5,
          "phrase": "Expect showers Monday morning",
          "category": "rain"
     },
     "forecasts": [
          {
               "date": "2025-09-01T07:00:00+02:00",
               "temperature": {
                    "minimum": {
                         "value": 12.9,
                         "unit": "C",
                         "unitType": 17
                    },
                    "maximum": {
                         "value": 22.4,
                         "unit": "C",
                         "unitType": 17
                    }
               },
               "realFeelTemperature": {
                    "minimum": {
                         "value": 12.2,
                         "unit": "C",
                         "unitType": 17
                    },
                    "maximum

### Get hourly forecast

In [9]:
print(f"Hourly Forecast for {address}:\n")
hourly_forecast = weather_client.get_hourly_forecast(lat, lon, duration=24)

print("24-Hour Forecast:")
print(json.dumps(hourly_forecast, indent=5))

Hourly Forecast for Paris, France:

24-Hour Forecast:
{
     "forecasts": [
          {
               "date": "2025-09-01T11:00:00+02:00",
               "iconCode": 3,
               "iconPhrase": "Partly sunny",
               "hasPrecipitation": false,
               "isDaylight": true,
               "temperature": {
                    "value": 19.2,
                    "unit": "C",
                    "unitType": 17
               },
               "realFeelTemperature": {
                    "value": 20.1,
                    "unit": "C",
                    "unitType": 17
               },
               "wetBulbTemperature": {
                    "value": 16.1,
                    "unit": "C",
                    "unitType": 17
               },
               "dewPoint": {
                    "value": 13.8,
                    "unit": "C",
                    "unitType": 17
               },
               "wind": {
                    "direction": {
                        

In [10]:
print(f"Hourly Forecast for {address}:\n")
hourly_forecast = weather_client.get_hourly_forecast(lat, lon, duration=24)

print("24-Hour Forecast:")
print(json.dumps(hourly_forecast, indent=5))

Hourly Forecast for Paris, France:

24-Hour Forecast:
{
     "forecasts": [
          {
               "date": "2025-09-01T11:00:00+02:00",
               "iconCode": 3,
               "iconPhrase": "Partly sunny",
               "hasPrecipitation": false,
               "isDaylight": true,
               "temperature": {
                    "value": 19.2,
                    "unit": "C",
                    "unitType": 17
               },
               "realFeelTemperature": {
                    "value": 20.1,
                    "unit": "C",
                    "unitType": 17
               },
               "wetBulbTemperature": {
                    "value": 16.1,
                    "unit": "C",
                    "unitType": 17
               },
               "dewPoint": {
                    "value": 13.8,
                    "unit": "C",
                    "unitType": 17
               },
               "wind": {
                    "direction": {
                        